# Goal

The objective of this notebook is to demonstrate and visualize the computation of the pareto frontier.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd


from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory
from mlos.OptimizerEvaluationTools.SyntheticFunctions.Hypersphere import Hypersphere

from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory, bayesian_optimizer_config_store
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Optimizers.ParetoFrontier import ParetoFrontier

from mlos.Spaces import ContinuousDimension, DiscreteDimension, Point, SimpleHypergrid

hypersphere_radius = 10

objective_function_config = Point(
    implementation=Hypersphere.__name__,
    hypersphere_config=Point(
        num_objectives=2,
        minimize='none',
        radius=hypersphere_radius
    )
)

objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config=objective_function_config)
optimization_problem = objective_function.default_optimization_problem
optimizer = BayesianOptimizerFactory().create_local_optimizer(
    optimization_problem=optimization_problem,
    optimizer_config=bayesian_optimizer_config_store.get_config_by_name("default_multi_objective_optimizer_config")
)

01/30/2021 04:03:15 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 106 -        _create_estimators() ] Creating 10 estimators. Tree config: {
  "criterion": "mse",
  "splitter": "best",
  "max_depth": 0,
  "min_samples_split": 2,
  "min_samples_leaf": 3,
  "min_weight_fraction_leaf": 0,
  "max_features": "auto",
  "max_leaf_nodes": 0,
  "min_impurity_decrease": 0,
  "ccp_alpha": 0,
  "min_samples_to_fit": 10,
  "n_new_samples_before_refit": 10
}. Request id: 0.47138267737279993
01/30/2021 04:03:15 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel with the input_space:   Name: estimator_0_input_space
  Dimensions:
    spherical_coordinates___theta0: [0.00, 1.57]
    spherical_coordinates___radius: [0.00, 10.00]
    spherical_coordinates___theta1: [0.00, 1.57]
01/30/2021 04:03:15 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomFores

01/30/2021 04:03:16 -   BayesianOptimizerFactory -    INFO - [HomogeneousRandomForestRegressionModel.py: 120 -        _create_estimators() ] Creating DecissionTreeRegressionModel with the input_space:   Name: estimator_9_input_space
  Dimensions:
    spherical_coordinates___theta0: [0.00, 1.57]
    spherical_coordinates___theta1: [0.00, 1.57]
    contains_context: {False}


In [3]:
optimizer.optimizer_config

{
  "surrogate_model_implementation": "MultiObjectiveHomogeneousRandomForest",
  "experiment_designer_implementation": "ParallelExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 10,
  "homogeneous_random_forest_regression_model_config.n_estimators": 10,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 0.7,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 0.7,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.criterion": "mse",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.splitter": "best",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.max_depth": 0,
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.min_s

In [4]:
import numpy as np

# Let's create the true pareto frontier so that we can plot it.
#
objective_function.parameter_space.dimensions
theta0 = objective_function.parameter_space['theta0'].linspace(20)
theta1 = objective_function.parameter_space['theta1'].linspace(20)
theta0, theta1 = np.meshgrid(theta0, theta1)
optimum_params_df = pd.DataFrame({
    'theta0': theta0.flatten(),
    'theta1': theta1.flatten(),
})
optimum_params_df['radius'] = hypersphere_radius
optimum_objectives_df = objective_function.evaluate_dataframe(optimum_params_df)
true_pareto_frontier = ParetoFrontier(optimization_problem, optimum_objectives_df)
true_pareto_df = true_pareto_frontier.pareto_df

In [5]:
import random

max_num_pending_suggestions = 10
pending_suggestions = []

tentative_pareto_dfs_over_time = []
pareto_dfs_over_time = []

In [ ]:
num_iterations = 100
for i in range(num_iterations):
    print(i)
    config = optimizer.suggest()
    pending_suggestions.append(config)
    optimizer.add_pending_suggestion(config)
    
    if len(pending_suggestions) >= max_num_pending_suggestions:
        config = pending_suggestions.pop(random.randint(0, len(pending_suggestions) - 1))
        value = objective_function.evaluate_point(config)
        optimizer.register(config.to_dataframe(), value.to_dataframe())
        
    tentative_pareto_dfs_over_time.append(optimizer.experiment_designer._tentative_pareto_frontier.pareto_df)
    pareto_dfs_over_time.append(optimizer.pareto_frontier.pareto_df)
    
for config in pending_suggestions:
    value = objective_function.evaluate_point(config)
    optimizer.register(config.to_dataframe(), value.to_dataframe())
    tentative_pareto_dfs_over_time.append(optimizer.experiment_designer._tentative_pareto_frontier.pareto_df)
    pareto_dfs_over_time.append(optimizer.pareto_frontier.pareto_df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [9]:
for config in pending_suggestions:
    value = objective_function.evaluate_point(config)
    optimizer.register(config.to_dataframe(), value.to_dataframe())
    tentative_pareto_dfs_over_time.append(optimizer.experiment_designer._tentative_pareto_frontier.pareto_df)
    pareto_dfs_over_time.append(optimizer.pareto_frontier.pareto_df)

params_df, objectives_df, context_df = optimizer.get_all_observations()
objectives_over_time = [objectives_df[:i] for i in objectives_df.index]

# Let's add a leading and trailing row to the frontier to make the volume clearer.
#
augmented_pareto_dfs_over_time = []
for pareto_df in pareto_dfs_over_time:
    max_y0 = pareto_df['y0'].max()
    max_y1 = pareto_df['y1'].max()
    last_row = pd.DataFrame({'y0': [0], 'y1': [pareto_df['y1'].max()]})
    first_row = pd.DataFrame({'y0': [pareto_df['y0'].max()], 'y1': [0]})
    augmented_pareto_df = pd.concat([first_row, pareto_df, last_row], ignore_index=True)
    augmented_pareto_dfs_over_time.append(augmented_pareto_df)

pareto_dfs_over_time = augmented_pareto_dfs_over_time

augmented_pareto_dfs_over_time = []
for pareto_df in tentative_pareto_dfs_over_time:
    max_y0 = pareto_df['y0'].max()
    max_y1 = pareto_df['y1'].max()
    last_row = pd.DataFrame({'y0': [0], 'y1': [pareto_df['y1'].max()]})
    first_row = pd.DataFrame({'y0': [pareto_df['y0'].max()], 'y1': [0]})
    augmented_pareto_df = pd.concat([first_row, pareto_df, last_row], ignore_index=True)
    augmented_pareto_dfs_over_time.append(augmented_pareto_df)

tentative_pareto_dfs_over_time = augmented_pareto_dfs_over_time

import plotly.graph_objects as go



# Create figure
fig = go.Figure(
    data=[
        go.Scatter(
            x=objectives_over_time[0]['y0'],
            y=objectives_over_time[0]['y1'],
            mode="markers",
            marker=dict(color="blue", size=10),
            name='observed objectives'
        ),
        go.Scatter(
            x=true_pareto_df['y0'],
            y=true_pareto_df['y1'],
            name='true pareto'
            #mode="markers",
            #marker=dict(color="red", size=10)
        )
    ],
    layout=go.Layout(
        xaxis=dict(range=[0, 30], autorange=False, zeroline=False),
        yaxis=dict(range=[0, 30], autorange=False, zeroline=False),
        title_text="Pareto over time",
        hovermode="closest",
        updatemenus=[dict(type="buttons", buttons=[dict(label="Play", method="animate", args=[None])])],
        width=700,
        height=600
    ),
    frames=[
        go.Frame(
            data=[
                #go.Scatter(
                #    x=true_pareto_df['y0'],
                #    y=true_pareto_df['y1'],
                #    name='true pareto',
                #    mode="lines",
                #    #marker=dict(color="red", size=10)
                #),
                go.Scatter(
                    x=pareto_dfs_over_time[i]['y0'],
                    y=pareto_dfs_over_time[i]['y1'],
                    name='estimated pareto',
                    mode='lines+markers'
                ),
                go.Scatter(
                    x=tentative_pareto_dfs_over_time[i]['y0'],
                    y=tentative_pareto_dfs_over_time[i]['y1'],
                    name='tentative pareto',
                    mode='lines+markers'
                )
            ]
        )
        for i in range(len(tentative_pareto_dfs_over_time))
    ]
)

fig.show()